In [ ]:
import fastf1
import pandas as pd
import os

# Enable FastF1 cache
fastf1.Cache.enable_cache('../cache')

# Output directory
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

# Seasons to process
seasons = [2021, 2022, 2023, 2024, 2025]

for season in seasons:
    print(f"Processing season {season}...")
    season_data = []

    try:
        season_schedule = fastf1.get_event_schedule(season, force_ergast=True)
    except Exception as e:
        print(f"Failed to load schedule for season {season}: {e}")
        continue

    for _, event in season_schedule.iterrows():
        round_num = event['RoundNumber']
        race_name = event['EventName']

        print(f"  Processing Round {round_num} - {race_name}...")

        try:
            # Qualifying
            quali = fastf1.get_session(season, round_num, 'Q')
            quali.load()

            pole_row = quali.results[quali.results['Position'] == 1]
            if pole_row.empty:
                continue

            pole_driver = pole_row['Abbreviation'].values[0]

            # Race
            race = fastf1.get_session(season, round_num, 'R')
            race.load()

            winner_row = race.results[race.results['Position'] == 1]
            if winner_row.empty:
                continue

            race_winner = winner_row['Abbreviation'].values[0]

            # Pole finish position
            pole_finish_row = race.results[race.results['Abbreviation'] == pole_driver]
            pole_finish = (
                pole_finish_row['Position'].values[0]
                if not pole_finish_row.empty else None
            )

            # Lap 1 position
            lap1_position = None
            try:
                laps = race.laps
                lap1 = laps[
                    (laps['Driver'] == pole_driver) &
                    (laps['LapNumber'] == 1)
                ]['Position'].values

                if len(lap1) > 0:
                    lap1_position = lap1[0]
            except:
                pass

            season_data.append({
                'Season': season,
                'Round': round_num,
                'Circuit': race_name,
                'PoleDriver': pole_driver,
                'RaceWinner': race_winner,
                'PoleFinishPosition': pole_finish,
                'Lap1Position': lap1_position,
                'WonFromPole': pole_driver == race_winner
            })

        except Exception as e:
            print(f"    Failed Round {round_num}: {e}")
            continue

    df_season = pd.DataFrame(season_data)
    file_path = f"{output_dir}poles_{season}.csv"
    df_season.to_csv(file_path, index=False)
    print(f"✅ Saved {file_path}")

print("✅ All seasons processed.")
